# GANs Traning

In [1]:
from tensorflow import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
import os
import sys 
import warnings
warnings.filterwarnings("ignore")

In [2]:
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense,Flatten, Input ,Dropout,Reshape
from tensorflow.keras.layers import BatchNormalization, Activation , ZeroPadding2D
from tensorflow.keras.optimizers import Adam
from keras.layers.advanced_activations import LeakyReLU ,ReLU
from keras.layers.convolutional import UpSampling2D, Conv2D

In [3]:
from keras.datasets import mnist

In [4]:
class Gans():
    def __init__(self):
        self.img_rows = 28
        self.img_cols =28
        self.channels =1
        self.img_shape =(self.img_rows , self.img_cols, self.channels )
        self.latent_dims =100

        optimizer = Adam(0.0002,0.5)
        
        #Build and Complile discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss="binary_crossentropy" , metrics =['accuracy'], optimizer = optimizer)


        #Build the Generators
        self.generator = self.build_generator()

        #input to generators
        z =Input(shape=(self.latent_dims,))
        img = self.generator(z)

        #now will train discriminator only 
        self.discriminator.trainable = False
    
        #discriminator take generated image as input and detemine the validity
        validity =self.discriminator(img)
    
        # stacking both model
        self.combined = Model(z,validity)
        
        #compile both model
        # Trains the generator to fool the discriminator
        self.combined.compile(loss="binary_crossentropy",optimizer = optimizer)
    
    
    def build_generator(self):
        model = Sequential()
        #layers_1
        model.add(Dense(256, input_dim = self.latent_dims))
        model.add(LeakyReLU(alpha = 0.2))
        model.add(BatchNormalization(momentum=0.8))
        #layers_2
        model.add(Dense(512))
        model.add(LeakyReLU(alpha = 0.2 ))
        model.add(BatchNormalization(momentum =0.8))
        #layers_3
        model.add(Dense(1024))
        model.add(LeakyReLU(alpha =0.2))
        model.add(BatchNormalization(momentum = 0.8))
        
        model.add(Dense(np.prod(self.img_shape), activation='tanh'))
        model.add(Reshape(self.img_shape))
        
        model.summary()
        noise = Input(shape=(self.latent_dims))
        
        img =model(noise)
        
        return Model(noise,img)
        
        
    def build_discriminator(self):
        
        model = Sequential()
        #input_layers
        model.add(Flatten(input_shape = self.img_shape))
        #layer_1
        model.add(Dense(512))
        model.add(LeakyReLU(alpha = 0.2))
        #layer_2
        model.add(Dense(256))
        model.add(LeakyReLU(alpha = 0.2))
        #output_layer
        model.add(Dense(1, activation='sigmoid'))
        #model summary
        model.summary()
        
        img = Input(shape=self.img_shape)
        validity = model(img)

        return Model(img, validity)
    
    def train(self , epochs , batch_size = 128 , sample_interval=50):
        
        #load the dataset 
        (X_train, _),(_,_) = mnist.load_data()
        
        #reshape -1 to 1
        X_train = X_train/127.5 -1
        X_train = np.expand_dims(X_train, axis=3)
        
        #Adversarial Truth
        valid =np.ones((batch_size ,1))
        fake  =np.zeros((batch_size,1))
        
        
        for epoch in range(epochs):
            
            #----- Train discriminator -------
            #---------------------------------
            #---------------------------------
            
            #Select a random batch images
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            imgs = X_train[idx]
            
            noise = np.random.normal(0,1,(batch_size, self.latent_dims))
            
            #generating new batch images
            gens_img  = self.generator(noise)
            
            #Train discriminator 
            d_loss_real =self.discriminator.train_on_batch(imgs , valid)
            d_loss_fake = self.discriminator.train_on_batch(gens_img , fake)
            d_loss = 0.5 * np.add(d_loss_real , d_loss_fake)
            
            #----- Train Generators-----------
            #---------------------------------
            #---------------------------------
            
            noise = np.random.normal(0, 1, (batch_size,self.latent_dims))
            
            #Train the generators
            g_loss =self.combined.train_on_batch(noise, valid)
            
            #Plot the progress 
            print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))
            
            # If at save interval => save generated image samples
            if epoch % sample_interval == 0:
                self.sample_images(epoch)
                
    def sample_images(self, epoch):
        r, c = 5, 5
        noise = np.random.normal(0, 1, (r * c, self.latent_dims))
        gen_imgs = self.generator.predict(noise)

        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5

        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
                axs[i,j].axis('off')
                cnt += 1
        fig.savefig("images/%d.png" % epoch)
        plt.close()
            

In [8]:
if __name__ == '__main__':
    gan = Gans()
    gan.train(epochs=15000, batch_size=32, sample_interval=500)

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_3 (Flatten)         (None, 784)               0         
                                                                 
 dense_21 (Dense)            (None, 512)               401920    
                                                                 
 leaky_re_lu_15 (LeakyReLU)  (None, 512)               0         
                                                                 
 dense_22 (Dense)            (None, 256)               131328    
                                                                 
 leaky_re_lu_16 (LeakyReLU)  (None, 256)               0         
                                                                 
 dense_23 (Dense)            (None, 1)                 257       
                                                                 
Total params: 533,505
Trainable params: 533,505
Non-tr

91 [D loss: 0.008679, acc.: 100.00%] [G loss: 4.568930]
92 [D loss: 0.009429, acc.: 100.00%] [G loss: 4.618530]
93 [D loss: 0.011569, acc.: 100.00%] [G loss: 4.554133]
94 [D loss: 0.011160, acc.: 100.00%] [G loss: 4.650434]
95 [D loss: 0.013304, acc.: 100.00%] [G loss: 4.559005]
96 [D loss: 0.010352, acc.: 100.00%] [G loss: 4.622860]
97 [D loss: 0.010105, acc.: 100.00%] [G loss: 4.535788]
98 [D loss: 0.010040, acc.: 100.00%] [G loss: 4.512580]
99 [D loss: 0.014787, acc.: 100.00%] [G loss: 4.636339]
100 [D loss: 0.012515, acc.: 100.00%] [G loss: 4.449138]
101 [D loss: 0.014412, acc.: 100.00%] [G loss: 4.534746]
102 [D loss: 0.020488, acc.: 100.00%] [G loss: 4.916183]
103 [D loss: 0.013104, acc.: 100.00%] [G loss: 4.731406]
104 [D loss: 0.012916, acc.: 100.00%] [G loss: 4.525829]
105 [D loss: 0.015259, acc.: 100.00%] [G loss: 4.718676]
106 [D loss: 0.014349, acc.: 100.00%] [G loss: 4.835748]
107 [D loss: 0.016243, acc.: 100.00%] [G loss: 4.885096]
108 [D loss: 0.018312, acc.: 100.00%] [G

239 [D loss: 0.325277, acc.: 76.56%] [G loss: 4.137752]
240 [D loss: 0.396578, acc.: 87.50%] [G loss: 2.183382]
241 [D loss: 0.161710, acc.: 95.31%] [G loss: 3.245139]
242 [D loss: 0.294447, acc.: 87.50%] [G loss: 2.790662]
243 [D loss: 0.433981, acc.: 71.88%] [G loss: 3.235155]
244 [D loss: 0.589174, acc.: 65.62%] [G loss: 2.275358]
245 [D loss: 0.260374, acc.: 92.19%] [G loss: 3.424513]
246 [D loss: 0.768030, acc.: 60.94%] [G loss: 1.928757]
247 [D loss: 0.216625, acc.: 96.88%] [G loss: 3.339880]
248 [D loss: 0.949171, acc.: 46.88%] [G loss: 1.211870]
249 [D loss: 0.355477, acc.: 81.25%] [G loss: 2.949973]
250 [D loss: 0.323101, acc.: 89.06%] [G loss: 2.913844]
251 [D loss: 0.357867, acc.: 81.25%] [G loss: 2.525365]
252 [D loss: 0.435923, acc.: 81.25%] [G loss: 2.408335]
253 [D loss: 0.776288, acc.: 59.38%] [G loss: 1.616141]
254 [D loss: 0.289640, acc.: 89.06%] [G loss: 2.453857]
255 [D loss: 0.673003, acc.: 62.50%] [G loss: 1.685220]
256 [D loss: 0.321414, acc.: 85.94%] [G loss: 2.

389 [D loss: 0.701149, acc.: 45.31%] [G loss: 0.662557]
390 [D loss: 0.687741, acc.: 50.00%] [G loss: 0.641087]
391 [D loss: 0.680831, acc.: 48.44%] [G loss: 0.622981]
392 [D loss: 0.670090, acc.: 50.00%] [G loss: 0.637078]
393 [D loss: 0.673104, acc.: 50.00%] [G loss: 0.643857]
394 [D loss: 0.658257, acc.: 50.00%] [G loss: 0.655043]
395 [D loss: 0.664996, acc.: 50.00%] [G loss: 0.642039]
396 [D loss: 0.666796, acc.: 46.88%] [G loss: 0.643292]
397 [D loss: 0.694461, acc.: 43.75%] [G loss: 0.646540]
398 [D loss: 0.675977, acc.: 45.31%] [G loss: 0.649134]
399 [D loss: 0.678544, acc.: 46.88%] [G loss: 0.656874]
400 [D loss: 0.652629, acc.: 50.00%] [G loss: 0.652742]
401 [D loss: 0.684298, acc.: 46.88%] [G loss: 0.648152]
402 [D loss: 0.667553, acc.: 45.31%] [G loss: 0.635460]
403 [D loss: 0.666582, acc.: 48.44%] [G loss: 0.621934]
404 [D loss: 0.654383, acc.: 48.44%] [G loss: 0.649301]
405 [D loss: 0.656646, acc.: 50.00%] [G loss: 0.658359]
406 [D loss: 0.675625, acc.: 43.75%] [G loss: 0.

541 [D loss: 0.628274, acc.: 71.88%] [G loss: 0.720170]
542 [D loss: 0.615484, acc.: 64.06%] [G loss: 0.727428]
543 [D loss: 0.625773, acc.: 62.50%] [G loss: 0.734808]
544 [D loss: 0.584407, acc.: 78.12%] [G loss: 0.745710]
545 [D loss: 0.617118, acc.: 65.62%] [G loss: 0.769540]
546 [D loss: 0.592692, acc.: 70.31%] [G loss: 0.786313]
547 [D loss: 0.665172, acc.: 53.12%] [G loss: 0.777394]
548 [D loss: 0.617972, acc.: 65.62%] [G loss: 0.763231]
549 [D loss: 0.652480, acc.: 62.50%] [G loss: 0.774592]
550 [D loss: 0.622120, acc.: 71.88%] [G loss: 0.769314]
551 [D loss: 0.628327, acc.: 65.62%] [G loss: 0.775643]
552 [D loss: 0.618368, acc.: 68.75%] [G loss: 0.747715]
553 [D loss: 0.639025, acc.: 68.75%] [G loss: 0.759593]
554 [D loss: 0.616503, acc.: 70.31%] [G loss: 0.754930]
555 [D loss: 0.610185, acc.: 67.19%] [G loss: 0.756571]
556 [D loss: 0.621116, acc.: 64.06%] [G loss: 0.751037]
557 [D loss: 0.620392, acc.: 60.94%] [G loss: 0.757290]
558 [D loss: 0.629728, acc.: 67.19%] [G loss: 0.

...

14371 [D loss: 0.697289, acc.: 53.12%] [G loss: 0.948998]
14372 [D loss: 0.609751, acc.: 65.62%] [G loss: 0.976858]
14373 [D loss: 0.609318, acc.: 70.31%] [G loss: 1.010935]
14374 [D loss: 0.651719, acc.: 64.06%] [G loss: 1.055044]
14375 [D loss: 0.580242, acc.: 70.31%] [G loss: 0.947763]
14376 [D loss: 0.610287, acc.: 68.75%] [G loss: 0.855663]
14377 [D loss: 0.691714, acc.: 56.25%] [G loss: 1.019449]
14378 [D loss: 0.655728, acc.: 59.38%] [G loss: 0.963783]
14379 [D loss: 0.710117, acc.: 53.12%] [G loss: 1.061478]
14380 [D loss: 0.570581, acc.: 73.44%] [G loss: 0.924891]
14381 [D loss: 0.677371, acc.: 54.69%] [G loss: 0.934862]
14382 [D loss: 0.596057, acc.: 65.62%] [G loss: 1.020143]
14383 [D loss: 0.612604, acc.: 67.19%] [G loss: 0.931865]
14384 [D loss: 0.646364, acc.: 62.50%] [G loss: 0.893095]
14385 [D loss: 0.655758, acc.: 65.62%] [G loss: 0.972206]
14386 [D loss: 0.677920, acc.: 62.50%] [G loss: 0.902552]
14387 [D loss: 0.685323, acc.: 54.69%] [G loss: 0.913993]
14388 [D loss:

14515 [D loss: 0.615918, acc.: 64.06%] [G loss: 0.851326]
14516 [D loss: 0.665874, acc.: 53.12%] [G loss: 0.886481]
14517 [D loss: 0.628232, acc.: 57.81%] [G loss: 0.884757]
14518 [D loss: 0.628686, acc.: 57.81%] [G loss: 1.029873]
14519 [D loss: 0.639268, acc.: 60.94%] [G loss: 0.958144]
14520 [D loss: 0.663999, acc.: 64.06%] [G loss: 1.000790]
14521 [D loss: 0.664519, acc.: 64.06%] [G loss: 0.979301]
14522 [D loss: 0.655432, acc.: 54.69%] [G loss: 0.964686]
14523 [D loss: 0.706073, acc.: 54.69%] [G loss: 1.010110]
14524 [D loss: 0.619044, acc.: 59.38%] [G loss: 0.991585]
14525 [D loss: 0.638299, acc.: 65.62%] [G loss: 0.979213]
14526 [D loss: 0.648790, acc.: 56.25%] [G loss: 1.070917]
14527 [D loss: 0.624736, acc.: 70.31%] [G loss: 0.966150]
14528 [D loss: 0.654450, acc.: 62.50%] [G loss: 0.968298]
14529 [D loss: 0.703076, acc.: 50.00%] [G loss: 0.938675]
14530 [D loss: 0.641365, acc.: 68.75%] [G loss: 0.975049]
14531 [D loss: 0.665220, acc.: 64.06%] [G loss: 0.968941]
14532 [D loss:

14660 [D loss: 0.641736, acc.: 64.06%] [G loss: 0.930963]
14661 [D loss: 0.671700, acc.: 62.50%] [G loss: 0.935588]
14662 [D loss: 0.599885, acc.: 67.19%] [G loss: 0.970897]
14663 [D loss: 0.667265, acc.: 65.62%] [G loss: 0.966255]
14664 [D loss: 0.668040, acc.: 60.94%] [G loss: 1.020049]
14665 [D loss: 0.629772, acc.: 67.19%] [G loss: 1.053014]
14666 [D loss: 0.690064, acc.: 59.38%] [G loss: 1.046722]
14667 [D loss: 0.658442, acc.: 57.81%] [G loss: 1.021430]
14668 [D loss: 0.645496, acc.: 62.50%] [G loss: 1.026593]
14669 [D loss: 0.669654, acc.: 65.62%] [G loss: 0.867136]
14670 [D loss: 0.718903, acc.: 56.25%] [G loss: 0.917001]
14671 [D loss: 0.644326, acc.: 57.81%] [G loss: 1.032412]
14672 [D loss: 0.642156, acc.: 60.94%] [G loss: 0.979269]
14673 [D loss: 0.607828, acc.: 68.75%] [G loss: 1.051666]
14674 [D loss: 0.628677, acc.: 62.50%] [G loss: 1.071790]
14675 [D loss: 0.690208, acc.: 57.81%] [G loss: 1.050016]
14676 [D loss: 0.656544, acc.: 56.25%] [G loss: 1.003367]
14677 [D loss:

14802 [D loss: 0.634969, acc.: 67.19%] [G loss: 0.959935]
14803 [D loss: 0.764831, acc.: 48.44%] [G loss: 0.857537]
14804 [D loss: 0.621809, acc.: 64.06%] [G loss: 1.016493]
14805 [D loss: 0.711501, acc.: 50.00%] [G loss: 0.967101]
14806 [D loss: 0.553275, acc.: 76.56%] [G loss: 1.030179]
14807 [D loss: 0.745111, acc.: 48.44%] [G loss: 0.962076]
14808 [D loss: 0.630167, acc.: 60.94%] [G loss: 0.950427]
14809 [D loss: 0.655480, acc.: 60.94%] [G loss: 1.053757]
14810 [D loss: 0.637528, acc.: 59.38%] [G loss: 0.991060]
14811 [D loss: 0.617683, acc.: 65.62%] [G loss: 1.078340]
14812 [D loss: 0.590648, acc.: 73.44%] [G loss: 0.840986]
14813 [D loss: 0.675669, acc.: 64.06%] [G loss: 0.798547]
14814 [D loss: 0.665622, acc.: 65.62%] [G loss: 1.075281]
14815 [D loss: 0.611747, acc.: 67.19%] [G loss: 0.967346]
14816 [D loss: 0.687265, acc.: 57.81%] [G loss: 0.985903]
14817 [D loss: 0.588261, acc.: 65.62%] [G loss: 1.010907]
14818 [D loss: 0.598360, acc.: 65.62%] [G loss: 0.911276]
14819 [D loss:

14948 [D loss: 0.674385, acc.: 53.12%] [G loss: 0.986757]
14949 [D loss: 0.636186, acc.: 67.19%] [G loss: 0.942900]
14950 [D loss: 0.619651, acc.: 70.31%] [G loss: 1.086105]
14951 [D loss: 0.607113, acc.: 64.06%] [G loss: 0.981356]
14952 [D loss: 0.690125, acc.: 56.25%] [G loss: 0.996492]
14953 [D loss: 0.565409, acc.: 78.12%] [G loss: 1.002575]
14954 [D loss: 0.679939, acc.: 56.25%] [G loss: 1.048946]
14955 [D loss: 0.528103, acc.: 75.00%] [G loss: 1.018645]
14956 [D loss: 0.764804, acc.: 51.56%] [G loss: 0.934667]
14957 [D loss: 0.682411, acc.: 56.25%] [G loss: 0.974955]
14958 [D loss: 0.665057, acc.: 54.69%] [G loss: 0.971033]
14959 [D loss: 0.658609, acc.: 54.69%] [G loss: 0.977504]
14960 [D loss: 0.634459, acc.: 67.19%] [G loss: 0.995885]
14961 [D loss: 0.669384, acc.: 60.94%] [G loss: 1.071512]
14962 [D loss: 0.632777, acc.: 62.50%] [G loss: 1.077748]
14963 [D loss: 0.682665, acc.: 62.50%] [G loss: 1.044850]
14964 [D loss: 0.633352, acc.: 65.62%] [G loss: 1.000526]
14965 [D loss: